In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import gc
import cartopy.feature as cfeature
ds = xr.open_dataset("../NC/compare.nc")  
names = ["agri", "irrig_agri", "grain", "econ", "grassland"]
years = list(range(2010, 2101, 10))   # 2010,2020,...,2100

In [2]:
from matplotlib.colors import LinearSegmentedColormap

# Panoply ACT colormap recreated
panoply_act_colors = [
    (0.0,   (0, 0, 128/255)),   # dark blue
    (0.125, (0, 0, 1)),
    (0.25,  (0, 0.7, 1)),       # cyan
    (0.375, (0.7, 0.9, 1)),     # light cyan
    (0.5,   (1, 1, 1)),         # white
    (0.625, (1, 1, 0)),         # yellow
    (0.75,  (1, 0.5, 0)),       # orange
    (0.875, (1, 0, 0)),         # red
    (1.0,   (0.5, 0, 0))        # dark red
]

panoply_act = LinearSegmentedColormap.from_list("panoply_act", panoply_act_colors)

diff_colors = [
    (0.0,   (0/255, 0/255, 70/255)),    # very dark blue
    (0.25,  (0/255, 60/255, 255/255)),  # saturated blue
    (0.5,   (0.7, 0.7, 0.7)),           # darker gray
    (0.75,  (255/255, 140/255, 0/255)), # strong orange
    (1.0,   (180/255, 0/255, 0/255))    # dark red
]

panoply_act_diff = LinearSegmentedColormap.from_list("panoply_act_diff", diff_colors)


In [5]:
names = ["agri", "irrig_agri", "grain", "econ", "grassland", "foerst"]
names = ["forest"]
years = list(range(2010, 2101, 10))   # 2010,2020,...,2100
yearsall = years.append(2005)
spectitle = "Forest" 
speclabel = "fraction of grid cell area"
lon = ds.lon
lat = ds.lat
plt.ioff()

In [ ]:


def plot_map_fast(data, mode_name, filename, year, vmin=0, vmax=1, cmap=panoply_act):

    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    im = ax.pcolormesh(
        lon,lat,data.values,
        cmap=cmap,vmin=vmin,vmax=vmax,
        shading="auto",transform=ccrs.PlateCarree() )

    ax.coastlines()
    ax.set_title(f"{spectitle} distribution in {mode_name} mode {year}", fontsize=14)

    cax = fig.add_axes([0.15, 0.08, 0.7, 0.03])
    cbar = fig.colorbar(im, cax=cax, orientation="horizontal")
    cbar.set_label(f"{spectitle}{speclabel}")
    cbar.ax.xaxis.set_label_position('top')

    fig.savefig(f"../plot/alone/{filename}", dpi=600, bbox_inches="tight")
    plt.close(fig)
    gc.collect()

for name in names:
    var_basin  = f"basin_{name}"
    var_region = f"region_{name}"
    for year in years:
        time = f"{year}-01-01"
        # basin
        da_basin = ds[var_basin].sel(time=time)
        plot_map_fast(
            data=da_basin,
            mode_name="basin",
            filename=f"{name}_basin_{year}.png",
            year=year)
        # region
        da_region = ds[var_region].sel(time=time)
        plot_map_fast(
            data=da_region,
            mode_name="region",
            filename=f"{name}_region_{year}.png",
            year=year)
    print(f"Finished {name}")

In [6]:
names = ["agri", "irrig_agri", "grain", "econ", "grassland"]
names = ["forest"]
years = list(range(2010, 2101, 10))   # 2010,2020,...,2100
years.append(2005)
for name in names:
    var_basin  = f"basin_{name}"
    var_region = f"region_{name}"

    mode_basin  = "basin"
    mode_region = "region"

    for year in years:
        time = f"{year}-01-01"

        da_basin  = ds[var_basin].sel(time=time)
        da_region = ds[var_region].sel(time=time)

        diff = da_region - da_basin

        fig = plt.figure(figsize=(12, 6))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

        im = ax.pcolormesh(
        lon,lat,diff,
        transform=ccrs.PlateCarree(),   # 数据仍然是lat/lon坐标
        cmap=panoply_act_diff,
        vmin=-0.4,vmax=0.4)

        ax.coastlines()
        ax.add_feature(cfeature.BORDERS.with_scale("50m"), linewidth=0.5)
        gl = ax.gridlines(draw_labels=False, linewidth=0.4, color='black', alpha=0.6)

        cbar = plt.colorbar(im, ax=ax, orientation="horizontal", pad=0.075, shrink=0.5)
        cbar.set_label(f"{name} fraction diff of grid cell area")
        cbar.ax.xaxis.set_label_position('top')

        plt.title(f"{name} diff ({mode_region} - {mode_basin}) : {name} {year}", fontsize=14)

        outname = f"{name}_{mode_region}-{mode_basin}_{year}.png"
        plt.savefig(f"../plot/diff/{outname}", dpi=1000, bbox_inches="tight")

        plt.close(fig)

    print(f"Finished diff maps of: {name}")


Finished diff maps of: forest
